In [35]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

In [101]:
from keras.layers import Dense, Dropout
from keras import Input, Sequential#, Model
import keras

In [3]:
# Read recipe inputs
census_income_features_train = dataiku.Dataset("census_income_features_train")
train = census_income_features_train.get_dataframe()

# Prepare Data

Split training data into train and validation sets (75/25 split). Drop unnecessary columns.

In [4]:
# features = ['weeks_worked_in_year', 'male', 'capital_gains']
drop_cols = ['target_column', 'instance_weight', 'num_employed_by_employer']
X = train.drop(drop_cols, axis=1)#[features]
y = train['target_column']
X.shape, y.shape

((199523, 212), (199523,))

In [5]:
train_proportion = 0.75
train_size = int(train_proportion*len(X))
print(train_size)

149642


In [6]:
# X_norm, y = np.array((X - X.mean())/X.std()), np.array(y)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((149642, 212), (149642,), (49881, 212), (49881,))

# Machine Learning: Training

### Logistic Regression and Tree-Based Ensemble Models

In [28]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
print("Logistic Regression Train Accuracy:", lr.score(X_train, y_train))
print("Logistic Regression Validation Accuracy:", lr.score(X_val, y_val))

/data/saas-instances/saas_d55tu7o0/data/code-envs/python/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Train Accuracy: 0.9517916093075474
Logistic Regression Validation Accuracy: 0.9514845331889898


In [63]:
rf = RandomForestClassifier(max_depth=20, n_estimators=100, random_state=0)
rf.fit(X_train, y_train)
print("Random Forest Train Accuracy:", rf.score(X_train, y_train))
print("Random Forest Validation Accuracy:", rf.score(X_val, y_val))

Random Forest Train Accuracy: 0.9656112588711725
Random Forest Validation Accuracy: 0.9523064894448788


In [64]:
gb = GradientBoostingClassifier(max_depth=10, n_estimators=20, random_state=0)
gb.fit(X_train, y_train)
print("Gradient Boosting Train Accuracy:", gb.score(X_train, y_train))
print("Gradient Boosting Validation Accuracy:", gb.score(X_val, y_val))

Random Forest Train Accuracy: 0.9640207963005039
Random Forest Validation Accuracy: 0.9537699725346325


In [65]:
# # Tune Parameters of Gradient Boosting Classifier
# for md in range(5, 50, 5):
#     clf = GradientBoostingClassifier(max_depth=md, random_state=0)
#     clf.fit(X_train, y_train)
#     print("Training and Validation Accuracy for ")
#     print(clf.score(X_train, y_train))
#     print(clf.score(X_val, y_val))

### Fully-Connected Neural Network (1 Hidden Layer)

In [10]:
keras.backend.clear_session()
model = Sequential()
model.add(Dense(8, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/data/saas-instances/saas_d55tu7o0/data/code-envs/python/py36/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:532: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  append_fn(tensor_proto, proto_values)
/data/saas-instances/saas_d55tu7o0/data/code-envs/python/py36/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:497: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
/data/saas-instances/saas_d55tu7o0/data/code-envs/python/py36/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [11]:
model.fit(X, y, batch_size=10, epochs=1000, validation_split=0.25)

Train on 149642 samples, validate on 49881 samples
Epoch 1/1000
149642/149642 [==============================] - 30s 199us/step - loss: 0.5456 - acc: 0.9353 - val_loss: 0.5228 - val_acc: 0.9382
Epoch 2/1000
149642/149642 [==============================] - 30s 198us/step - loss: 0.5019 - acc: 0.9353 - val_loss: 0.4201 - val_acc: 0.9422
Epoch 3/1000
149642/149642 [==============================] - 29s 195us/step - loss: 0.4576 - acc: 0.9385 - val_loss: 0.4167 - val_acc: 0.9398
Epoch 4/1000
149642/149642 [==============================] - 29s 193us/step - loss: 0.3491 - acc: 0.9399 - val_loss: 0.3438 - val_acc: 0.9398
Epoch 5/1000
149642/149642 [==============================] - 29s 196us/step - loss: 0.3192 - acc: 0.9405 - val_loss: 0.1432 - val_acc: 0.9455
Epoch 6/1000
149642/149642 [==============================] - 29s 193us/step - loss: 0.1436 - acc: 0.9423 - val_loss: 0.1442 - val_acc: 0.9401
Epoch 7/1000
149642/149642 [==============================] - 28s 190us/step - loss: 0.1435

149642/149642 [==============================] - 30s 204us/step - loss: 0.1298 - acc: 0.9487 - val_loss: 0.1377 - val_acc: 0.9430
Epoch 58/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1297 - acc: 0.9489 - val_loss: 0.1347 - val_acc: 0.9462
Epoch 59/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1298 - acc: 0.9488 - val_loss: 0.1313 - val_acc: 0.9501
Epoch 60/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1294 - acc: 0.9492 - val_loss: 0.1321 - val_acc: 0.9487
Epoch 61/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1300 - acc: 0.9492 - val_loss: 0.1311 - val_acc: 0.9479
Epoch 62/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1290 - acc: 0.9492 - val_loss: 0.1329 - val_acc: 0.9489
Epoch 63/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1296 - acc: 0.9487 - val_loss: 0.1305 - val_acc: 0.9516
Epoch 

Epoch 114/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1291 - acc: 0.9499 - val_loss: 0.1343 - val_acc: 0.9513
Epoch 115/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1294 - acc: 0.9499 - val_loss: 0.1377 - val_acc: 0.9479
Epoch 116/1000
149642/149642 [==============================] - 34s 226us/step - loss: 0.1281 - acc: 0.9496 - val_loss: 0.1318 - val_acc: 0.9498
Epoch 117/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1283 - acc: 0.9495 - val_loss: 0.1356 - val_acc: 0.9495
Epoch 118/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1287 - acc: 0.9501 - val_loss: 0.1332 - val_acc: 0.9511
Epoch 119/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1284 - acc: 0.9505 - val_loss: 0.1290 - val_acc: 0.9509
Epoch 120/1000
149642/149642 [==============================] - 33s 223us/step - loss: 0.1282 - acc: 0.9505 - val_loss: 0.1417 - v

149642/149642 [==============================] - 31s 206us/step - loss: 0.1283 - acc: 0.9500 - val_loss: 0.1321 - val_acc: 0.9498
Epoch 171/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1281 - acc: 0.9500 - val_loss: 0.1301 - val_acc: 0.9512
Epoch 172/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1283 - acc: 0.9498 - val_loss: 0.1354 - val_acc: 0.9508
Epoch 173/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1284 - acc: 0.9502 - val_loss: 0.1342 - val_acc: 0.9492
Epoch 174/1000
149642/149642 [==============================] - 31s 206us/step - loss: 0.1281 - acc: 0.9501 - val_loss: 0.1360 - val_acc: 0.9488
Epoch 175/1000
149642/149642 [==============================] - 31s 206us/step - loss: 0.1289 - acc: 0.9500 - val_loss: 0.1386 - val_acc: 0.9497
Epoch 176/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1278 - acc: 0.9502 - val_loss: 0.1345 - val_acc: 0.9473


149642/149642 [==============================] - 31s 204us/step - loss: 0.1286 - acc: 0.9500 - val_loss: 0.1329 - val_acc: 0.9499
Epoch 227/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1279 - acc: 0.9504 - val_loss: 0.1384 - val_acc: 0.9489
Epoch 228/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1280 - acc: 0.9507 - val_loss: 0.1319 - val_acc: 0.9515
Epoch 229/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1272 - acc: 0.9506 - val_loss: 0.1379 - val_acc: 0.9497
Epoch 230/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1279 - acc: 0.9503 - val_loss: 0.1310 - val_acc: 0.9505
Epoch 231/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1282 - acc: 0.9504 - val_loss: 0.1387 - val_acc: 0.9484
Epoch 232/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1275 - acc: 0.9509 - val_loss: 0.1395 - val_acc: 0.9493


...

149642/149642 [==============================] - 31s 205us/step - loss: 0.1275 - acc: 0.9513 - val_loss: 0.1343 - val_acc: 0.9487
Epoch 731/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1282 - acc: 0.9510 - val_loss: 0.1362 - val_acc: 0.9485
Epoch 732/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1276 - acc: 0.9510 - val_loss: 0.1328 - val_acc: 0.9498
Epoch 733/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1270 - acc: 0.9510 - val_loss: 0.1323 - val_acc: 0.9495
Epoch 734/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1274 - acc: 0.9513 - val_loss: 0.1338 - val_acc: 0.9505
Epoch 735/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1275 - acc: 0.9514 - val_loss: 0.1323 - val_acc: 0.9510
Epoch 736/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1278 - acc: 0.9512 - val_loss: 0.1308 - val_acc: 0.9516


149642/149642 [==============================] - 31s 205us/step - loss: 0.1276 - acc: 0.9516 - val_loss: 0.1321 - val_acc: 0.9487
Epoch 787/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1299 - acc: 0.9509 - val_loss: 0.1336 - val_acc: 0.9506
Epoch 788/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1275 - acc: 0.9514 - val_loss: 0.1307 - val_acc: 0.9515
Epoch 789/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1276 - acc: 0.9512 - val_loss: 0.1419 - val_acc: 0.9510
Epoch 790/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1271 - acc: 0.9513 - val_loss: 0.1451 - val_acc: 0.9460
Epoch 791/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1275 - acc: 0.9510 - val_loss: 0.1320 - val_acc: 0.9507
Epoch 792/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1278 - acc: 0.9514 - val_loss: 0.1335 - val_acc: 0.9487


149642/149642 [==============================] - 28s 186us/step - loss: 0.1276 - acc: 0.9515 - val_loss: 0.1321 - val_acc: 0.9500
Epoch 843/1000
149642/149642 [==============================] - 27s 179us/step - loss: 0.1272 - acc: 0.9516 - val_loss: 0.1323 - val_acc: 0.9513
Epoch 844/1000
149642/149642 [==============================] - 27s 179us/step - loss: 0.1288 - acc: 0.9510 - val_loss: 0.1716 - val_acc: 0.9492
Epoch 845/1000
149642/149642 [==============================] - 25s 166us/step - loss: 0.1276 - acc: 0.9517 - val_loss: 0.1357 - val_acc: 0.9487
Epoch 846/1000
149642/149642 [==============================] - 30s 199us/step - loss: 0.1282 - acc: 0.9512 - val_loss: 0.1375 - val_acc: 0.9500
Epoch 847/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1273 - acc: 0.9514 - val_loss: 0.1326 - val_acc: 0.9513
Epoch 848/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1277 - acc: 0.9513 - val_loss: 0.1303 - val_acc: 0.9511


149642/149642 [==============================] - 30s 203us/step - loss: 0.1270 - acc: 0.9516 - val_loss: 0.1312 - val_acc: 0.9506
Epoch 899/1000
149642/149642 [==============================] - 30s 204us/step - loss: 0.1270 - acc: 0.9516 - val_loss: 0.1359 - val_acc: 0.9474
Epoch 900/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1290 - acc: 0.9514 - val_loss: 0.1325 - val_acc: 0.9510
Epoch 901/1000
149642/149642 [==============================] - 31s 206us/step - loss: 0.1266 - acc: 0.9518 - val_loss: 0.1356 - val_acc: 0.9506
Epoch 902/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1277 - acc: 0.9511 - val_loss: 0.1389 - val_acc: 0.9508
Epoch 903/1000
149642/149642 [==============================] - 31s 206us/step - loss: 0.1276 - acc: 0.9514 - val_loss: 0.1350 - val_acc: 0.9513
Epoch 904/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1279 - acc: 0.9514 - val_loss: 0.1393 - val_acc: 0.9506


149642/149642 [==============================] - 31s 204us/step - loss: 0.1274 - acc: 0.9514 - val_loss: 0.1335 - val_acc: 0.9498
Epoch 955/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1276 - acc: 0.9515 - val_loss: 0.1358 - val_acc: 0.9510
Epoch 956/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1277 - acc: 0.9512 - val_loss: 0.1374 - val_acc: 0.9511
Epoch 957/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1276 - acc: 0.9515 - val_loss: 0.1331 - val_acc: 0.9509
Epoch 958/1000
149642/149642 [==============================] - 31s 204us/step - loss: 0.1308 - acc: 0.9509 - val_loss: 0.1397 - val_acc: 0.9503
Epoch 959/1000
149642/149642 [==============================] - 31s 205us/step - loss: 0.1280 - acc: 0.9511 - val_loss: 0.1428 - val_acc: 0.9497
Epoch 960/1000
149642/149642 [==============================] - 30s 203us/step - loss: 0.1294 - acc: 0.9516 - val_loss: 0.1321 - val_acc: 0.9512


# Machine Learning: Model Evaluation

Check accuracy as well as various other metrics for candidate models on test set. NOTE: ONLY DO THIS AFTER TUNING PARAMETERS ON TRAINING/VALIDATION SET AND DECIDING ON FINAL CANDIDATE MODEL.

In [14]:
census_income_features_test = dataiku.Dataset("census_income_features_test")
test = census_income_features_test.get_dataframe()

In [15]:
X_test = test.drop(drop_cols, axis=1)#[features]
y_test = test['target_column']
X.shape, y.shape

((199523, 212), (199523,))

In [33]:
print("Logistic Regression Test Accuracy:", lr.score(X_test, y_test))
print("Random Forest Test Accuracy:", rf.score(X_test, y_test))
print("Gradient Boosting Test Accuracy:", gb.score(X_test, y_test))
results = model.evaluate(X_test, y_test, batch_size=10)
print("Neural Network Test Accuracy:", results[1])

Logistic Regression Test Accuracy: 0.9521461077364126
Random Forest Test Accuracy: 0.9526372767185902
Gradient Boosting Test Accuracy: 0.9539303542430986
99762/99762 [==============================] - 6s 55us/step
Neural Network Test Accuracy: 0.9516148366784899


In [51]:
def auc(y_pred, y_test):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    return metrics.auc(fpr, tpr)
def f1(y_pred, y_test):
    return metrics.f1_score(y_pred, y_test, average='macro')

In [34]:
lr_pred = lr.predict(X_test)
rf_pred = rf.predict(X_test)
gb_pred = gb.predict(X_test)
nn_pred = model.predict(X_test)

In [57]:
nn_r = np.round(nn_pred)

In [58]:
print("Logistic Regression AUC:", auc(lr_pred, y_test))
print("Random Forest AUC:", auc(rf_pred, y_test))
print("Gradient Boosting AUC:", auc(gb_pred, y_test))
print("Neural Network AUC:", auc(nn_r, y_test))

Logistic Regression AUC: 0.6821401445751922
Random Forest AUC: 0.6516043184491833
Gradient Boosting AUC: 0.6846009316472751
Neural Network AUC: 0.7066158437003792


In [59]:
print("Logistic Regression F1-Score:", f1(lr_pred, y_test))
print("Random Forest F1-Score:", f1(rf_pred, y_test))
print("Gradient Boosting F1-Score:", f1(gb_pred, y_test))
print("Neural Network F1-Score:", f1(nn_r, y_test))

Logistic Regression F1-Score: 0.7335088656585602
Random Forest F1-Score: 0.710828771678815
Gradient Boosting F1-Score: 0.7398113589225578
Neural Network F1-Score: 0.7485104862868084


In [69]:
# Generate Feature Importances for Best Tree-Based Model
gb_feature_imp = gb.feature_importances_

In [72]:
gb_feature_imp.shape

(199523, 212)

In [90]:
cat_cols = ['class_of_worker', 'education', 'edu_inst_last_wk', 'marital_status', 
            'industry', 'occupation', 'race', 'member_labor_union', 'unemployment_reason', 
            'employment_status', 'tax_filer_status', 'previous_residence_region', 'household_summary', 
            'change_in_msa', 'change_in_reg', 'live_in_this_house_1_year_ago', 'family_members_under_18', 
            'country_of_birth_father', 'country_of_birth_mother', 'country_of_birth_self', 'citizenship', 
            'own_business_or_self_employed', 'veterans_benefits']
x_cols = list(X.columns)
feature_imp_dict = {}

for i in range(len(x_cols)):
    for c in cat_cols:
        if c in x_cols[i]:
            x_cols[i] = c
    feature_imp_dict[x_cols[i]] = gb_feature_imp[i]

In [100]:
gb_feature_imp_df = pd.DataFrame(pd.Series(feature_imp_dict).sort_values(ascending=False)).reset_index()
gb_feature_imp_df

,index,0
0,capital_gains,2.439243e-01
1,dividends_from_stocks,1.368124e-01
2,weeks_worked_in_year,1.195087e-01
3,male,7.747848e-02
4,age,6.310153e-02
5,capital_losses,5.045051e-02
6,wage_per_hour,1.009618e-02
7,tax_filer_status,5.842207e-03
8,household_summary,2.463272e-03
9,race,1.710253e-03
